## Impact of Seasonal Natural Gas Consumption on Air Quality
- Analyze how seasonal variations in natural gas consumption influence air quality.

In [1]:
# Importing dependencies
#Python SQL Toolkit and ORM
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, text

#Pandas and Bokeh libraries
import pandas as pd
import calendar
from bokeh.plotting import figure, show, save, output_file
from bokeh.models import ColumnDataSource,Select, Div,Legend, LegendItem
from bokeh.io import output_file, output_notebook
from bokeh.layouts import column, layout, gridplot, row

#### Fetching data from our database and converting it to pandas DataFrame

In [2]:
#Create Database Connection
engine = create_engine("sqlite:///../Resources/new_aqi_ngsDB.sqlite", echo=False)

In [3]:
#Refelect Database into ORM Classes
Base = automap_base()
Base.prepare(autoload_with=engine)
Base.classes.keys()

['combined_df']

In [4]:
#map the combined_df class

combined_table_aqi_ng = Base.classes.combined_df

#create session
session = Session(engine)

In [5]:
#get a list of column names and types
inspector = inspect(engine)
columns = inspector.get_columns('combined_df')

for col in columns:
    print(col['name'], col['type'])

id INTEGER
state_name TEXT
state_id TEXT
month INTEGER
AQI INTEGER
lat REAL
lng REAL
density INTEGER
population INTEGER
Category TEXT
Commercial Consumption REAL
Delivered to Consumers REAL
Electric Power Consumption REAL
Industrial Consumption REAL
Residential Consumption REAL
Vehicle Fuel Consumption REAL
total REAL


In [6]:
#Query the data from our table
data_query = session.query(combined_table_aqi_ng.id,combined_table_aqi_ng.state_id,combined_table_aqi_ng.month,combined_table_aqi_ng.AQI,combined_table_aqi_ng.state_name,
                           combined_table_aqi_ng.lat,combined_table_aqi_ng.lng,combined_table_aqi_ng.Category,combined_table_aqi_ng.population,combined_table_aqi_ng.total ).all()

for item in data_query:
    print(item)

(0, 'AL', 1, 37, 'Alabama', 33.3173, -86.8356, 'Good', 52337600, 133551.0)
(1, 'AL', 2, 38, 'Alabama', 33.2297, -86.8105, 'Good', 48578333, 114933.0)
(2, 'AL', 3, 43, 'Alabama', 33.1279, -86.8814, 'Good', 60388837, 103717.0)
(3, 'AL', 4, 47, 'Alabama', 33.1279, -86.8814, 'Good', 58440810, 96171.0)
(4, 'AL', 5, 44, 'Alabama', 33.1231, -86.8856, 'Good', 60374774, 101999.0)
(5, 'AL', 6, 38, 'Alabama', 33.1279, -86.8814, 'Good', 58440810, 115579.0)
(6, 'AL', 7, 39, 'Alabama', 33.1369, -86.8778, 'Good', 60362259, 119120.0)
(7, 'AL', 8, 42, 'Alabama', 33.1135, -86.8943, 'Good', 60346648, 124022.0)
(8, 'AL', 9, 37, 'Alabama', 33.1279, -86.8814, 'Good', 58440810, 109530.0)
(9, 'AL', 10, 37, 'Alabama', 33.146, -86.8742, 'Good', 60335681, 109041.0)
(10, 'AL', 11, 38, 'Alabama', 33.2614, -86.8268, 'Good', 52910666, 115678.0)
(11, 'AL', 12, 34, 'Alabama', 33.3429, -86.7456, 'Good', 55791569, 119300.0)
(12, 'AK', 1, 58, 'Alaska', 61.4802, -143.6454, 'Moderate', 10699991, 15243.0)
(13, 'AK', 2, 42, 

In [7]:
#converting the query result into a pandas DataFrame
query_list = []

for row in data_query:
    query_dics={
        'id':row.id,
        'state':row.state_name,
        'month':row.month,
        'AQI':row.AQI,
        'lat':row.lat,
        'lng':row.lng,
        'category':row.Category,
        'population':row.population,
        'natural_gas_consumption':row.total
        }
    query_list.append(query_dics)

df = pd.DataFrame(query_list)
df.head()

,id,state,month,AQI,lat,lng,category,population,natural_gas_consumption
0,0,Alabama,1,37,33.3173,-86.8356,Good,52337600,133551.0
1,1,Alabama,2,38,33.2297,-86.8105,Good,48578333,114933.0
2,2,Alabama,3,43,33.1279,-86.8814,Good,60388837,103717.0
3,3,Alabama,4,47,33.1279,-86.8814,Good,58440810,96171.0
4,4,Alabama,5,44,33.1231,-86.8856,Good,60374774,101999.0


In [8]:
filtered_df = df[df['state'].isin(['Alaska','Minnesota','New York','Idaho','Arkansas','New Jersey','California','Florida','Texas','District of Columbia'])].reset_index()

#,'New Jersey','Alabama','Florida','Texas','District of Columbia'

filtered_df.head()

,index,id,state,month,AQI,lat,lng,category,population,natural_gas_consumption
0,12,12,Alaska,1,58,61.4802,-143.6454,Moderate,10699991,15243.0
1,13,13,Alaska,2,42,61.4802,-143.6454,Good,9664508,15370.0
2,14,14,Alaska,3,29,61.4802,-143.6454,Good,10699991,15715.0
3,15,15,Alaska,4,31,61.6964,-144.3220,Good,10162236,11956.0
4,16,16,Alaska,5,24,61.4802,-143.6454,Good,10699991,9288.0


In [9]:
#Define a function for mapping months to seasons

def seasons(month):
    if month in [12,1,2]:
        return 'Winter'
    elif month in [6,7,8]:
        return 'Summer'
    elif month in [3,4,5]:
        return 'Spring'
    else:
        return 'Fall'
    
#applying the functino to create a new season column in our dataframe for each months numeric value
filtered_df['season'] = filtered_df['month'].apply(seasons)

filtered_df.head(25)

,index,id,state,month,AQI,lat,lng,category,population,natural_gas_consumption,season
0,12,12,Alaska,1,58,61.4802,-143.6454,Moderate,10699991,15243.0,Winter
1,13,13,Alaska,2,42,61.4802,-143.6454,Good,9664508,15370.0,Winter
2,14,14,Alaska,3,29,61.4802,-143.6454,Good,10699991,15715.0,Spring
3,15,15,Alaska,4,31,61.6964,-144.3220,Good,10162236,11956.0,Spring
4,16,16,Alaska,5,24,61.4802,-143.6454,Good,10699991,9288.0,Spring
5,17,17,Alaska,6,21,61.4802,-143.6454,Good,10354830,7352.0,Summer
6,18,18,Alaska,7,29,61.4802,-143.6454,Good,10699991,7429.0,Summer
7,19,19,Alaska,8,19,61.5467,-143.8536,Good,10635793,8279.0,Summer
8,20,20,Alaska,9,18,61.4802,-143.6454,Good,10354830,8563.0,Fall
9,21,21,Alaska,10,26,61.4802,-143.6454,Good,10699991,12262.0,Fall


### Seasonal Analysis of Air Quality and Natural Gas Consumption
- Objective: Examine how air quality and natural gas consumptioin varies with seasons.


In [10]:
seasonal_aqi = filtered_df.groupby(['state','season']).agg({
    'AQI':'mean',
    'natural_gas_consumption':'mean'
}).reset_index()
#seasonal_aqi = filtered_df.groupby(['state','season'])

seasonal_aqi['AQI'] = round(seasonal_aqi['AQI'],0)
seasonal_aqi['natural_gas_consumption'] = round(seasonal_aqi['natural_gas_consumption'],0)

seasonal_aqi.head(20)

,state,season,AQI,natural_gas_consumption
0,Alaska,Fall,28.0,16090.0
1,Alaska,Spring,28.0,12320.0
2,Alaska,Summer,23.0,7687.0
3,Alaska,Winter,50.0,20438.0
4,Arkansas,Fall,40.0,56627.0
5,Arkansas,Spring,40.0,55430.0
6,Arkansas,Summer,44.0,54133.0
7,Arkansas,Winter,34.0,64056.0
8,California,Fall,61.0,327319.0
9,California,Spring,49.0,310298.0


#### Visualizing Seasonal Analysis of Air Quality and Natural Gas Consumption for 10 states

In [14]:
#Visualizing Seasonal Analysis of Air Quality and Natural Gas Consumption for 10 states
#Using Bokeh Library Line and scatter plots

from bokeh.palettes import Category20

src = ColumnDataSource(seasonal_aqi)

states = seasonal_aqi['state'].unique()
seasons = seasonal_aqi['season'].unique().tolist()

# Use a larger color palette for each state lines on our plot
palette = Category20[len(states)] if len(states) <= 20 else Category20[20]

# Create a Natural Gas figure to plot
gas_p = figure( x_range=seasons, 
           y_axis_label='natural_gas_consumption', 
           width=1000, 
           height=400)
gas_p.title.text = "Seasonal Trends of Natural Gas Consumption"

# Create a AQI figure to plot
aqi_p = figure( x_range=seasons, 
           y_axis_label='AQI ', 
           width=1000, 
           height=400)
aqi_p.title.text = "Seasonal Trends of Air Quality Index "

# List to store legend items (list of top 10 states)
legend_items = []

#Iterate over each states list to create plots for each state data
for i, state in enumerate(states):
    state_data=seasonal_aqi[seasonal_aqi['state']==state]
    source = ColumnDataSource(state_data)

    #Natural Gas Graph
    gas_line = gas_p.line(x='season', y='natural_gas_consumption', source=source, color=palette[i], line_width=2)
    gas_scatter = gas_p.scatter(x='season', y='natural_gas_consumption', source=source, color=palette[i], size=8)

    #AQI Graph
    aqi_line = aqi_p.line(x='season', y='AQI', source=source, color=palette[i], line_width=2)
    aqi_scatter = aqi_p.scatter(x='season', y='AQI', source=source, color=palette[i], size=8)

     # Create a LegendItem for the state that includes both line and circle
    legend_item = LegendItem(label=state, renderers=[gas_line, gas_scatter,aqi_line,aqi_scatter])
    legend_items.append(legend_item)

# Add the legend to the plot
legend = Legend(items=legend_items)
gas_p.add_layout(legend, 'right')
aqi_p.add_layout(legend, 'right')
    
# Customize the Natural Gas plot
gas_p.legend.title="Click to hide the corresponding line"
gas_p.legend.location = 'top_right'
gas_p.legend.click_policy = 'hide'  # Allow toggling of state lines
gas_p.xaxis.axis_label = 'Seasons'
gas_p.yaxis.axis_label = 'Average Natural Gas Consumption'
gas_p.xgrid.grid_line_color = None
gas_p.ygrid.grid_line_dash = [6, 4]

# Customize the AQI  plot
aqi_p.legend.title="Click to hide the corresponding line"
aqi_p.legend.location = 'top_right'
aqi_p.legend.click_policy = 'hide'  # Allow toggling of state lines
#aqi_p.xaxis.axis_label = 'Season'
aqi_p.yaxis.axis_label = 'Average Air Quality Index'
aqi_p.xgrid.grid_line_color = None
aqi_p.ygrid.grid_line_dash = [6, 4]

# Create a Div widget to display the comparison text
div_text = Div(
    text="""
    <div style="text-align: center; background-color:lightgreen; padding: 10px;">
        <h2>Comparison of Air Quality Index against Natural Gas consumption over different seasons for top 10 States</h2>
    </div>
        """,
    width=2000,
    height=50
)

# Combine plot and div into a layout
layout = column(div_text,aqi_p,gas_p)

# Save the plot and layout as HTML
output_file("../../templates/seasonal_aqi_natural_gas_comparison_for_ten_states.html")
save(layout)

# Show the plot
show(layout)